In [138]:
using Pipe

parse_tile(line) = @pipe replace(line, r"([ns][ew]|e|w)" => s" \1 ") |> split(_, " "; keepempty=false)

parse_tile (generic function with 1 method)

In [139]:
tiles, offsets, input = Dict(), Dict(
    "e"  => (-1,  1,  0),
    "w"  => ( 1, -1,  0),
    "sw" => ( 1,  0, -1),
    "se" => ( 0,  1, -1),
    "nw" => ( 0, -1,  1),
    "ne" => (-1,  0,  1)
), parse_tile.(open(collect∘eachline, "24.dat"))

tile_coords(moves) = reduce((a, m) -> broadcast(+, a, offsets[m]), moves; init=(0,0,0))
flip_tile(coords) = tiles[coords] = !get(tiles, coords, false)

flip_tile (generic function with 1 method)

In [140]:
flip_tile.(tile_coords.(input));

In [141]:
@show tiles |> values |> count

(tiles |> values) |> count = 465


465

In [142]:
function next_state(tiles, cc, next, recurse=true)
    v, s = get(tiles, cc, false), (broadcast(+, cc, o) for o in values(offsets))
    surr = count(map(x -> get(tiles, x, false), s))
    (v || surr == 2) && (next[cc] = !(v && (surr == 0 || surr > 2)))
    for a in (recurse ? s : ()); next_state(tiles, a, next, false) end
end

function all_next(n)
    curr, new = tiles, Dict()
    for i in 1:n
         for a in keys(curr); next_state(curr, a, new) end
         curr, new = new, Dict()
    end
    curr
end

all_next (generic function with 1 method)

In [143]:
@time all_next(100) |> values |> count |> println

4078
  2.526108 seconds (23.54 M allocations: 845.403 MiB, 4.93% gc time)
